## Imports

In [1]:
! pip3 install -r requirements.txt
import requests
from bs4 import BeautifulSoup
from time import sleep
import pickle
import redis
import json

    100% |████████████████████████████████| 71kB 209kB/s ta 0:00:01


## Helper functions

In [2]:
def get_pars(link, parser='html.parser'):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, parser)
    return soup

def get_sub_categories(soup,selector):
    # get sub categories
    sub_categories = soup.select(selector)

    list_of_subcategories = []
    for each_category in sub_categories:
        list_of_subcategories.append(['https://www.digikala.com' + each_category.get('href'), each_category.get_text()])
#         print('https://www.digikala.com' + each_category.get('href'), each_category.get_text())
    return list_of_subcategories

def check_url(link):
    if not 'digikala.com' in link:
        return 'https://digikala.com' + link
    else:
        return link

def get_product_pics(soup):
    list_of_product_pic = []
    product_images = soup.select('img')
    for each in product_images:
        if each.get('data-src') != None: 
            link = each.get('data-src')
            link_pos = link.find('?x') 
            if '.jpg' in link and not link[:link_pos] in list_of_product_pic:
                list_of_product_pic.append(link[:link_pos])
#                 print(link[:link_pos])

def get_similar_products(soup):
    similar_products = soup.select(similar_products_selector)
    list_of_similar_products = []
    for each_similar in similar_products:
        link = each_similar.get('href')
        if type(link) != type(None) and '/product/' in link and '/users/' not in link and 'twitter' not in link and 'facebook' not in link:
            if check_url(link) not in list_of_similar_products:
                list_of_similar_products.append(check_url(link))
#                 print(check_url(link))
    return list_of_similar_products

## Page related configs

In [3]:
categories_selector = "body header nav div div ul li ul li div div ul li a"
sub_categories_selector = "div article div section div ul li a"
product_in_subcategories_selector = "div div div div div article div ul li div div div div a"
similar_products_selector = 'div div div div a'

## DB related config

In [131]:
r = redis.Redis(host='localhost', port=6379, db=0)

## Main

In [5]:
# get categories
soup = get_pars("https://digikala.com")
categories = soup.select(categories_selector)
l =[]
list_of_categories = []
for each in categories:
    if each.get('href') not in l:
        l.append(each.get('href'))
        list_of_categories.append([each.get('href'),each.get_text().strip()])
del l
len(list_of_categories)

425

In [6]:
list_of_search_pages =[]
for each_category in list_of_categories:
    if '/main/' in each_category[0]:
        link = check_url(each_category[0])
        soup = get_pars(link)
        temp_sub = get_sub_categories(soup,sub_categories_selector)
        for i in temp_sub:
            list_of_search_pages.append(i[0])
    elif '/search' in each_category[0]:
        link = check_url(each_category[0])
        list_of_search_pages.append(link)
    else:
        print(each_category)

In [8]:
len(list_of_search_pages)

722

In [ ]:
modified_list_of_search_pages = []
for each in list_of_search_pages:
    for i in range(1,6):
        modified_list_of_search_pages.append(each + '?pageno={}&sortby=4'.format(i))
list_of_search_pages = modified_list_of_search_pages
del modified_list_of_search_pages

In [ ]:
list_of_search_pages

In [ ]:
for each_search_page in list_of_search_pages:
    soup = get_pars(each_search_page)
    products = soup.select(product_in_subcategories_selector)
    list_of_page_products = []
    for each_product in products:
        list_of_page_products.append(['https://www.digikala.com' + each_product.get('href'),each_product.get_text()])
        print('https://www.digikala.com' + each_product.get('href'),each_product.get_text())
    sleep(5)

In [132]:
with open('data.pkl','rb') as f:
    list_of_search_pages = pickle.load(f)

In [133]:
p1 = list_of_search_pages[0][30][0]
p1

'https://www.digikala.com/product/dkp-657703/هندزفری-بلوتوثی-هوکار-مدل-sport-rhs6-با-قابلیت-مکالمه'

In [140]:
for each_category in list_of_search_pages:
    for each_product in each_category:
        if type(r.get(each_product[0])) == type(None):
            r.set(each_product[0], json.dumps([each_product[1],get_similar_products(get_pars(each_product[0]))]))
            print(each_product)
            sleep(5)

In [ ]:
r.get('https://www.digikala.com/product/dkp-310914/هندزفری-مدل-eo-ig955')

In [118]:
r.set('foo', 'bar')

True

In [119]:
r.set('foo', 'bar2')

True

In [120]:
r.get('foo')

b'bar2'

In [102]:
r.delete('foo')

1

In [103]:
type(r.get('foo'))

NoneType

In [109]:
a = [1,[2,3,4,5,6,7]]


In [111]:
type(json.dumps(a))

str

In [124]:
r.flushdb()

True